In [12]:
import geneticalgorithm as ga
from typing import List, Tuple
import random
import numpy as np

In [13]:
def fitness_function(ing_list: List[int]) -> float:
    taste_sum = 0
    health_sum = 0

    health_sum = sum([x['health'] for x in ing_list])
    taste_sum = sum([x['taste'] for x in ing_list])
    # we prioritize health over taste
    return 0.7 * health_sum + 0.3 * taste_sum


import random

# Create a dummy list of dictionaries
dummy_data: List[dict] = [{'health': random.randint(1, 10), 'taste': random.randint(1, 10)} for _ in range(100)]

# print(dummy_data)
health_sum = sum([x['health'] for x in dummy_data])
taste_sum = sum([x['taste'] for x in dummy_data])

health_sum, taste_sum


assert fitness_function(dummy_data) == (health_sum * 0.7 + taste_sum * 0.3)

In [14]:
# sample random recipe from json

import json

with open('recipes_expanded.json') as f:
    recipes = json.load(f)

In [15]:
fitness_function(random.choice(recipes)["ingredients"])

35.1

# Fitness function

`f(x) = 0.7 health_score + 0.3 taste_score`

Should also add some check to have flour to be mandatory for example, spesific item to be determined



In [16]:
import random

def dummy_fitness_function(ing_list) -> float:
    return .7 * ing_list["health"] + .3 * ing_list["taste"]


dummy_discts = [{'index': i, 'health': random.randint(1, 10), 'taste': random.randint(1, 10)} for i in range(5)]

for i in dummy_discts:
    i["weight"] = dummy_fitness_function(i)

dummy_discts[0]['weight'] = 0.5

for i in dummy_discts:
    print(i)

random.choices(dummy_discts, weights=[x["weight"] for x in dummy_discts], k = 10)

{'index': 0, 'health': 6, 'taste': 9, 'weight': 0.5}
{'index': 1, 'health': 7, 'taste': 1, 'weight': 5.199999999999999}
{'index': 2, 'health': 5, 'taste': 1, 'weight': 3.8}
{'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1}
{'index': 4, 'health': 2, 'taste': 1, 'weight': 1.7}


[{'index': 1, 'health': 7, 'taste': 1, 'weight': 5.199999999999999},
 {'index': 2, 'health': 5, 'taste': 1, 'weight': 3.8},
 {'index': 2, 'health': 5, 'taste': 1, 'weight': 3.8},
 {'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1},
 {'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1},
 {'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1},
 {'index': 2, 'health': 5, 'taste': 1, 'weight': 3.8},
 {'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1},
 {'index': 2, 'health': 5, 'taste': 1, 'weight': 3.8},
 {'index': 3, 'health': 10, 'taste': 7, 'weight': 9.1}]

# Selection

To select the initial population (parents) we will use proportional selection. To create the weight for each normalize the fitnes for each recipe.

$F{weight} = F_i/F_{total}$

```python
def proportional_selection(population, selection_probabilities, num_selections=1):
    return random.choices(population, weights=selection_probabilities, k=num_selections)
```

Create a dictionary with `index` and ~~`normalised fitness`~~ (weights can be un-normalized)

**Example code**:
```python

import random

def dummy_fitness_function(ing_list) -> float:
    return .7 * ing_list["health"] + .3 * ing_list["taste"]


dummy_discts = [{'index': i, 'health': random.randint(1, 10), 'taste': random.randint(1, 10)} for i in range(5)]

for i in dummy_discts:
    i["weight"] = dummy_fitness_function(i)

dummy_discts[0]['weight'] = 0.5

for i in dummy_discts:
    print(i)

random.choices(dummy_discts, weights=[x["weight"] for x in dummy_discts], k = 10)
```


# Crossover
Available crossover choices:
* one point crossover
* uniform crossover

After crossover check if we have a duplicate and we sum their amounts.
```python

def merge_duplicate_ingredients(recipe: dict) -> dict:
    ingredient_dict = {}
    for ingredient in recipe["ingredients"]:
        ing_name = ingredient["ingredient"]
        if ing_name in ingredient_dict:
            ingredient_dict[ing_name]["amount"] += ingredient["amount"]
        else:
            ingredient_dict[ing_name] = ingredient

    recipe["ingredients"] = list(ingredient_dict.values())
    return recipe

# Example usage with test_dup
merged_recipe = merge_duplicate_ingredients(test_dup)
for i in merged_recipe["ingredients"]:
    print(i)
```

In [17]:
def check_duplicates(ingredients: List[dict]) -> bool:
    seen = set()
    for ingredient in ingredients:
        ingredient_tuple = tuple(ingredient.items())
        if ingredient_tuple in seen:
            return True
        seen.add(ingredient_tuple)
    return False

# Example usage with dummy_data
has_duplicates = check_duplicates(dummy_data)
print("Duplicates found:", has_duplicates)

Duplicates found: True


In [18]:
test_dup = {
    "index": 1,
    "name": "healthy-pumpkin-cookies",
    "url": "https://chocolatecoveredkatie.com/healthy-pumpkin-cookies",
    "ingredients": [
      {
        "ingredient": "quick oats",
        "amount": 180.0,
        "units": "g",
        "health": 9,
        "taste": 7
      },
      {
        "ingredient": "oat flour",
        "amount": 120.0,
        "units": "g",
        "health": 8,
        "taste": 7
      },
      {
        "ingredient": "sugar",
        "amount": 60.0,
        "units": "g",
        "health": 3,
        "taste": 9
      },
      {
        "ingredient": "chocolate chips",
        "amount": 45.0,
        "units": "g",
        "health": 5,
        "taste": 10
      },
      {
        "ingredient": "cinnamon",
        "amount": 1.25,
        "units": "g",
        "health": 7,
        "taste": 8
      },
      {
        "ingredient": "pumpkin pie spice",
        "amount": 1.25,
        "units": "g",
        "health": 6,
        "taste": 8
      },
      {
        "ingredient": "salt",
        "amount": 1.25,
        "units": "g",
        "health": 3,
        "taste": 9
      },
      {
        "ingredient": "baking soda",
        "amount": 1.25,
        "units": "g",
        "health": 3,
        "taste": 4
      },
      {
        "ingredient": "canned pumpkin",
        "amount": 80.0,
        "units": "g",
        "health": 9,
        "taste": 6
      },
      {
        "ingredient": "almond butter",
        "amount": 80.0,
        "units": "ml",
        "health": 8,
        "taste": 8
      },
      {
        "ingredient": "almond butter",
        "amount": 2.5,
        "units": "ml",
        "health": 5,
        "taste": 7
      }
    ]
  }

In [19]:
for i in test_dup["ingredients"]:
    print(i)

{'ingredient': 'quick oats', 'amount': 180.0, 'units': 'g', 'health': 9, 'taste': 7}
{'ingredient': 'oat flour', 'amount': 120.0, 'units': 'g', 'health': 8, 'taste': 7}
{'ingredient': 'sugar', 'amount': 60.0, 'units': 'g', 'health': 3, 'taste': 9}
{'ingredient': 'chocolate chips', 'amount': 45.0, 'units': 'g', 'health': 5, 'taste': 10}
{'ingredient': 'cinnamon', 'amount': 1.25, 'units': 'g', 'health': 7, 'taste': 8}
{'ingredient': 'pumpkin pie spice', 'amount': 1.25, 'units': 'g', 'health': 6, 'taste': 8}
{'ingredient': 'salt', 'amount': 1.25, 'units': 'g', 'health': 3, 'taste': 9}
{'ingredient': 'baking soda', 'amount': 1.25, 'units': 'g', 'health': 3, 'taste': 4}
{'ingredient': 'canned pumpkin', 'amount': 80.0, 'units': 'g', 'health': 9, 'taste': 6}
{'ingredient': 'almond butter', 'amount': 80.0, 'units': 'ml', 'health': 8, 'taste': 8}
{'ingredient': 'almond butter', 'amount': 2.5, 'units': 'ml', 'health': 5, 'taste': 7}


In [20]:
def merge_duplicate_ingredients(recipe: dict) -> dict:
    ingredient_dict = {}
    for ingredient in recipe["ingredients"]:
        ing_name = ingredient["ingredient"]
        if ing_name in ingredient_dict:
            ingredient_dict[ing_name]["amount"] += ingredient["amount"]
        else:
            ingredient_dict[ing_name] = ingredient

    recipe["ingredients"] = list(ingredient_dict.values())
    return recipe

# Example usage with test_dup
merged_recipe = merge_duplicate_ingredients(test_dup)
for i in merged_recipe["ingredients"]:
    print(i)

{'ingredient': 'quick oats', 'amount': 180.0, 'units': 'g', 'health': 9, 'taste': 7}
{'ingredient': 'oat flour', 'amount': 120.0, 'units': 'g', 'health': 8, 'taste': 7}
{'ingredient': 'sugar', 'amount': 60.0, 'units': 'g', 'health': 3, 'taste': 9}
{'ingredient': 'chocolate chips', 'amount': 45.0, 'units': 'g', 'health': 5, 'taste': 10}
{'ingredient': 'cinnamon', 'amount': 1.25, 'units': 'g', 'health': 7, 'taste': 8}
{'ingredient': 'pumpkin pie spice', 'amount': 1.25, 'units': 'g', 'health': 6, 'taste': 8}
{'ingredient': 'salt', 'amount': 1.25, 'units': 'g', 'health': 3, 'taste': 9}
{'ingredient': 'baking soda', 'amount': 1.25, 'units': 'g', 'health': 3, 'taste': 4}
{'ingredient': 'canned pumpkin', 'amount': 80.0, 'units': 'g', 'health': 9, 'taste': 6}
{'ingredient': 'almond butter', 'amount': 82.5, 'units': 'ml', 'health': 8, 'taste': 8}


# Mutation
Mutation types:
* amount of ingredient, select random ing, and scale its amount 0.25-2 e.g `amount = int(amount * random(0.25, 2))`
* change ingredient with another, if liquid change from liquids if solid from solids
* add ingredient (maybe tricky because we have to also add the amount for it)
* delete an ingredient

In [99]:
def mutate_recipe(recipe):
    # mutation_type = random.randint(0, 3)
    mutation_type = 3
    print("Mutation type:", mutation_type)
    # Mutation: Change ingredient amount
    if mutation_type == 0:
        i = random.randint(0, len(recipe['ingredients']) - 1)
        recipe['ingredients'][i] = recipe['ingredients'][i].copy()
        # print("Old amount:",  recipe['ingredients'][i]['amount'])
        recipe['ingredients'][i]['amount'] = max(1, recipe['ingredients'][i]['amount'] + recipe['ingredients'][i]['amount'] * 0.15)

        # print("New amount:", max(1, recipe['ingredients'][i]['amount'] + int(recipe['ingredients'][i]['amount'] * 0.15)))
        # print("Ingredient:", recipe['ingredients'][i]['ingredient'])

    # Mutation: Change one ingredient to another
    elif mutation_type == 1:
        j = random.randint(0, len(recipe['ingredients']) - 1)
        recipe['ingredients'][j] = recipe['ingredients'][j].copy()

        # Find a new ingredient that is different from the current one
        current_ingredient = recipe['ingredients'][j]-['ingredient']
        possible_substitutes = [ing for ing in unique_ingredients if ing['ingredient'] != current_ingredient]

        if possible_substitutes:
            new_ingredient = random.choice(possible_substitutes)
            recipe['ingredients'][j]['ingredient'] = new_ingredient['ingredient']

        # print("New ingredient:", recipe['ingredients'][j]['ingredient'])
        # print("Old ingredient:", current_ingredient)


    # Mutation: Addition of an ingredient
    elif mutation_type == 2:
        # new_ingredient = random.choice(unique_ingredients).copy()
        new_ingredient = random.choice(unique_inredients).copy()
        new_ingredient['amount'] = random.randint(1, 5)  # Random amount for the new ingredient
        # recipe['ingredients'].append(new_ingredient)
        # if the new ingredient is already in the recipe, add the amount to the existing ingredient and do not append a new ingredient in the recipe
        if new_ingredient['ingredient'] in [ing['ingredient'] for ing in recipe['ingredients']]:
            for ingredient in recipe['ingredients']:
                if ingredient['ingredient'] == new_ingredient['ingredient']:
                    ingredient['amount'] += new_ingredient['amount']
                    break
        else:
            recipe['ingredients'].append(new_ingredient)
        # print("New ingredient:", new_ingredient['ingredient'])
        

    # Mutation: Deletion of an ingredient
    elif mutation_type == 3:
        if len(recipe['ingredients']) > 1:
            deleted_ing = recipe['ingredients'].pop(random.randint(0, len(recipe['ingredients']) - 1))
        # print("Deleted ingredient:", deleted_ing['ingredient'])
    return recipe


mutated_recipe = mutate_recipe(recipe)
print(mutated_recipe)


Mutation type: 3
{'name': 'Healthy Pumpkin Cookies', 'ingredients': [{'ingredient': 'quick oats', 'amount': 3, 'unit': 'cup'}, {'ingredient': 'sugar', 'amount': 1, 'unit': 'tbsp'}]}
